** EXTRACCION DE DATOS **

In [ ]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime

In [3]:
import importlib.util
import sys
import os

script_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts/functions', 'limpieza.py'))

spec = importlib.util.spec_from_file_location("limpieza", script_path)
limpieza = importlib.util.module_from_spec(spec)
spec.loader.exec_module(limpieza)

In [2]:
# Versiones
print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

numpy==2.2.4
pandas==2.2.3
requests==2.32.3


In [5]:
# Constantes comunes
URL = "https://apidatos.ree.es" # URL base de la API
HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es",
    "User-Agent": "Mozilla/5.0"
}

# Diccionario de geo_ids y nombres de regiones
region_ids = {
    "8741": "peninsular",
    "8742": "canarias",
    "8743": "baleares",
    "8744": "ceuta",
    "8745": "melilla",
    "4": "andalucia",
    "5": "aragon",
    "6": "cantabria",
    "7": "castilla la mancha",
    "8": "castilla y leon",
    "9": "cataluña",
    "10": "pais vasco",
    "11": "principado de asturias",
    "13": "comunidad de madrid",
    "14": "comunidad de navarra",
    "15": "comunidad de valenciana",
    "16": "extremadura",
    "17": "galicia",
    "20": "la rioja",
    "21": "region de murcia"
}

In [6]:
filtro_balance = '/es/datos/balance/balance-electrico'

filtro_demanda = '/es/datos/demanda/evolucion'
filtro_general = '/es/datos/demanda/ire-general'
filtro_industria = '/es/datos/demanda/ire-industria'
filtro_servicios = '/es/datos/demanda/ire-servicios'

filtro_generacion = "/es/datos/generacion/estructura-generacion"
filtro_renovable = "/es/datos/generacion/evolucion-renovable-no-renovable"

In [3]:
def Balance_electrico(filtro_balance, URL, HEADERS):

    filtro_balance

    endpoint = f"{URL}/{filtro_balance}"

    # Inicializamos un DataFrame vacío para almacenar los datos
    df_balance = pd.DataFrame()

    # Iteramos sobe los años de 2019 a 2024 y extraemos los datos de balance
    # de cada región para cada año
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            data = response.json()
            included = data.get('included', [])

            registros = []
            # Extraemos los datos de balance
            # para cada grupo de energía
            # y cada indicador
            for grupo in included:
                group_name = grupo['attributes']['title']
                contenidos = grupo['attributes'].get('content', [])

                for item in contenidos:
                    indicador = item['attributes']['title']
                    valores = item['attributes'].get('values', [])
                    # Extraemos los valores para cada indicador
                    # y los almacenamos en el DataFrame
                    for punto in valores:
                        registros.append({
                            'fecha': punto['datetime'],
                            'valor': punto['value'],
                            'tipo': indicador,
                            'energia': group_name,
                            'region': region_name,
                        })
            # Convertimos los registros a un DataFrame
            # y lo concatenamos al DataFrame principal
            df_balance_year = pd.DataFrame(registros)
            df_balance = pd.concat([df_balance, df_balance_year], ignore_index=True)

    return df_balance

In [9]:
def demanda_evolucion(filtro_demanda, URL, HEADERS):

    filtro_demanda

    endpoint = f"{URL}/{filtro_demanda}"
    # Inicializamos un df vacío para almacenar los datos
    df_demanda = pd.DataFrame()
    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                # Extraer los valores de la serie
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'indicador': indicador,
                        'region': region_name,
                    })

            df_demanda_year = pd.DataFrame(registros)
            df_demanda = pd.concat([df_demanda, df_demanda_year], ignore_index=True)

    return df_demanda



In [10]:
def demanda_ire_general(filtro_general, URL, HEADERS):
    
    filtro_general

    endpoint = f"{URL}/{filtro_general}"

    # Inicializamos un DataFrame vacío
    df_ire_general = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                        
                    })

            df_ire_year = pd.DataFrame(registros)
            df_ire_general = pd.concat([df_ire_general, df_ire_year], ignore_index=True)

    return df_ire_general

In [11]:
def demanda_ire_industria(filtro_industria, URL, HEADERS):

    filtro_industria

    endpoint = f"{URL}/{filtro_industria}"

    # Inicializamos un DataFrame vacío
    df_ire_industria = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                    })

            df_ire_industria_year = pd.DataFrame(registros)
            df_ire_industria = pd.concat([df_ire_industria, df_ire_industria_year], ignore_index=True)

    return df_ire_industria

In [12]:
def demanda_ire_servicios(filtro_servicios, URL, HEADERS):

    filtro_servicios

    endpoint = f"{URL}/{filtro_servicios}"

    # Inicializamos un DataFrame vacío
    df_ire_servicios = pd.DataFrame()

    # Iteramos sobre los años y regiones
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "month"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "month"

        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code != 200:
                print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
                continue

            data = response.json()
            included = data.get('included', [])

            registros = []

            for serie in included:
                indicador = serie['attributes']['title']
                valores = serie['attributes'].get('values', [])
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'porcentaje': punto.get('percentage'),
                        'indicador': indicador,
                        'region': region_name,
                    })

            df_ire_servicios_year = pd.DataFrame(registros)
            df_ire_servicios = pd.concat([df_ire_servicios, df_ire_servicios_year], ignore_index=True)

    return df_ire_servicios

In [13]:
def generacion(filtro_generacion, URL, HEADERS):

    filtro_generacion

    endpoint = f"{URL}{filtro_generacion}"

    df_generacion = pd.DataFrame()
    ## Iterar sobre los años
    for year in range(2015, 2026):

        if year < 2025:
            start_date = f"{year}-01-01T00:00"
            end_date = f"{year}-12-31T23:59"
            TIME_TRUNC = "day"
        else:
            start_date = f"{year}-01-01T00:00"
            end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
            TIME_TRUNC = "day"

        # Iterar sobre los geo_ids y nombres de regiones
        # para cada año y geo_id
        for geo_id, region_name in region_ids.items():
            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
                "geo_id": geo_id
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                try:
                    data = response.json()
                    included = data.get('included', [])

                    if included:
                        registros = []

                        # Iterar sobre los grupos de datos en 'included'
                        for grupo in included:
                            group_name = grupo['attributes'].get('title', 'Desconocido')
                            group_type = grupo['attributes'].get('type', 'Desconocido')
                            contenidos = grupo['attributes'].get('values', [])

                            # Iterar sobre los valores de generación
                            for punto in contenidos:
                                registros.append({
                                    'fecha': punto.get('datetime', 'Desconocido'),
                                    'valor': punto.get('value', 0),
                                    'porcentaje': punto.get('percentage', 0),
                                    'indicador': group_name,
                                    'region': region_name,
                                    'tipo': group_type,
                                })

                        df_generacion_year = pd.DataFrame(registros)
                        df_generacion = pd.concat([df_generacion, df_generacion_year], ignore_index=True)

                except Exception as e:
                    print("Contenido de la respuesta:", response.text)

            else:
                print(f"Error en la solicitud para {region_name} {year}. Código: {response.status_code}")

    return df_generacion

In [14]:
def fronteras(URL, HEADERS):

    # Lista de países
    lista_paises = ['francia-frontera', 'portugal-frontera', 'marruecos-frontera', 'andorra-frontera']

    datos_intercambios = []

    for pais in lista_paises:
        # Endpoint para intercambios
        filtro_intercambio = f"/es/datos/intercambios/{pais}"
        endpoint = f"{URL}{filtro_intercambio}"
        # Iterar sobre los años y extraer datos de cada año
        # y cada país para el año 2015 al 2025
        for year in range(2015, 2026):

            if year < 2025:
                start_date = f"{year}-01-01T00:00"
                end_date = f"{year}-12-31T23:59"
                TIME_TRUNC = "day"
            else:
                start_date = f"{year}-01-01T00:00"
                end_date = datetime.now().strftime("%Y-%m-%dT%H:%M")
                TIME_TRUNC = "day"

            params = {
                "start_date": start_date,
                "end_date": end_date,
                "time_trunc": TIME_TRUNC,
            }

            response = requests.get(endpoint, headers=HEADERS, params=params)

            if response.status_code == 200:
                data = response.json()
                # Verificar si hay datos disponibles
                valores = data["included"][0]["attributes"].get("values", [])
                if valores:
                    df = pd.DataFrame(valores)
                    df["pais"] = pais
                    datos_intercambios.append(df)
            else:
                print(f"Error en la solicitud para {pais} en {year}. Código: {response.status_code}")

    # Unir todos los datos en un único DataFrame
    df_fronteras = pd.concat(datos_intercambios, ignore_index=True)

    return df_fronteras

In [15]:
def extraccion():
    """
    Función para extraer datos de la API de REE.
    """
    print('Extrayendo datos del Balance Eléctrico...')
    df_balance = Balance_electrico(filtro_balance, URL, HEADERS)
    
    print('Extrayendo datos de la Demanda Eléctrica...')
    df_demanda = demanda_evolucion(filtro_demanda, URL, HEADERS)
    print('Extrayendo datos de la IRE General...')
    df_ire_general = demanda_ire_general(filtro_general, URL, HEADERS)
    print('Extrayendo datos de la IRE Industria...')
    df_ire_industria = demanda_ire_industria(filtro_industria, URL, HEADERS)
    print('Extrayendo datos de la IRE Servicios...')
    df_ire_servicios = demanda_ire_servicios(filtro_servicios, URL, HEADERS)

    print('Extrayendo datos de la Generación Eléctrica...')
    df_generacion = generacion(filtro_generacion, URL, HEADERS)

    print('Extrayendo datos de los intercambios entre países...')
    df_fronteras = fronteras(URL, HEADERS)

    return {
        "balance": df_balance,
        "demanda": df_demanda,
        "ire_general": df_ire_general,
        "ire_industria": df_ire_industria,
        "ire_servicios": df_ire_servicios,
        "generacion": df_generacion,
        "fronteras": df_fronteras,
    }

In [ ]:
dataframes = extraccion() # Tarda aprox 17 minutos en hacer toda la extraccion de todos los datos

Extrayendo datos del Balance Eléctrico...
Extrayendo datos de la Demanda Eléctrica...
Extrayendo datos de la IRE General...
Extrayendo datos de la IRE Industria...
Extrayendo datos de la IRE Servicios...
Extrayendo datos de la Generación Eléctrica...
Extrayendo datos de los intercambios entre países...


** LIMPIEZA DE DATOS **

In [5]:
df_balance = dataframes['balance']
print('Balance:')
display(df_balance.head())
df_demanda = dataframes['demanda']
print('Demanda:')
display(df_demanda.head())
df_ire_general = dataframes['ire_general']
print('IRE General:')
display(df_ire_general.head())
df_ire_industria = dataframes['ire_industria']
print('IRE Industria:')
display(df_ire_industria.head())
df_ire_servicios = dataframes['ire_servicios']
print('IRE Servicios:')
display(df_ire_servicios.head())
df_generacion = dataframes['generacion']
print('Generacion:')
display(df_generacion.head())
df_fronteras = dataframes['fronteras']
print('Fronteras:')
display(df_fronteras.head())

NameError: name 'dataframes' is not defined

In [4]:
import importlib.util
import sys

script_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts/functions', 'limpieza.py'))

spec = importlib.util.spec_from_file_location("limpieza", script_path)
limpieza = importlib.util.module_from_spec(spec)
spec.loader.exec_module(limpieza)

df_balance_limpio = limpieza.limpieza_balance(df_balance)
df_demanda_limpia, df_ire_limpia = limpieza.limpieza_demanda(df_demanda, df_ire_general, df_ire_industria, df_ire_servicios)
df_generacion_limpia = limpieza.limpieza_generacion(df_generacion)
df_fronteras_limpias = limpieza.limpieza_fronteras(df_fronteras)

NameError: name 'df_balance' is not defined

In [38]:
print('Limpieza de datos:')
print('Balance:')
display(df_balance_limpio.head())
print('Demanda:')
display(df_demanda_limpia.head())
print('IRE:')
display(df_ire_limpia.head())
print('Generación:')
display(df_generacion_limpia.head())
print('Fronteras:')
display(df_fronteras_limpias.head())

Limpieza de datos:
Balance:


,fecha,valor,tipo,energia,region
0,2015-01-01,71.096101,Hidráulica,Renovable,peninsular
1,2015-01-02,101.890114,Hidráulica,Renovable,peninsular
2,2015-01-03,82.919726,Hidráulica,Renovable,peninsular
3,2015-01-04,69.056368,Hidráulica,Renovable,peninsular
4,2015-01-05,104.975898,Hidráulica,Renovable,peninsular


Demanda:


,fecha,valor,indicador,region
0,2014-12-31 23:00:00+00:00,603961.474,Demanda,peninsular
6205,2014-12-31 23:00:00+00:00,603961.474,Demanda,galicia
365,2014-12-31 23:00:00+00:00,603961.474,Demanda,canarias
5840,2014-12-31 23:00:00+00:00,603961.474,Demanda,extremadura
5475,2014-12-31 23:00:00+00:00,603961.474,Demanda,comunidad de valenciana


IRE:


,fecha,valor,porcentaje,indicador,region,tipo
0,2014-12-31 23:00:00+00:00,118.061000,0.538442,Índice general,peninsular,general
228,2014-12-31 23:00:00+00:00,2.945545,1.000000,Variación mensual corregida,melilla,industria
216,2014-12-31 23:00:00+00:00,2.329259,1.000000,Variación mensual,melilla,industria
204,2014-12-31 23:00:00+00:00,99.816000,0.451245,Índice industria corregido,melilla,industria
192,2014-12-31 23:00:00+00:00,119.056000,0.543953,Índice industria,melilla,industria


Generación:


,fecha,valor,porcentaje,indicador,region,tipo
0,2015-01-01,71.096101,0.114430,Hidráulica,peninsular,Renovable
1,2015-01-02,101.890114,0.147343,Hidráulica,peninsular,Renovable
2,2015-01-03,82.919726,0.117828,Hidráulica,peninsular,Renovable
3,2015-01-04,69.056368,0.101494,Hidráulica,peninsular,Renovable
4,2015-01-05,104.975898,0.145461,Hidráulica,peninsular,Renovable


Fronteras:


,valor,porcentaje,fecha,pais
0,-3.384261,0.122156,2015-01-01,francia-frontera
1,-0.060199,0.002045,2015-01-02,francia-frontera
2,-0.019245,0.000649,2015-01-03,francia-frontera
3,-0.031131,0.000995,2015-01-04,francia-frontera
4,-0.002958,0.000098,2015-01-05,francia-frontera


CARGAR A LA BASE DE DATOS

In [6]:
import sys
import os
import importlib.util

# Añadir la carpeta principal del proyecto a sys.path para que pueda encontrar el módulo 'scripts'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)


script_path = os.path.abspath(os.path.join(project_root, 'scripts/functions', 'carga.py'))

spec = importlib.util.spec_from_file_location("carga", script_path)
carga = importlib.util.module_from_spec(spec)
spec.loader.exec_module(carga)

In [55]:
carga_de_datos = carga.carga_de_datos(df_balance_limpio, df_generacion_limpia, df_fronteras_limpias, df_demanda_limpia, df_ire_limpia)

if carga_de_datos:
    print("Carga de datos exitosa.")
else:
    print("Error en la carga de datos.")

Cargando datos en la base de datos...
Conexión exitosa a la base de datos.
Carga de datos exitosa.
